# RAG Pipeline Demo for SAQ

Демонстрация работы RAG (Retrieval-Augmented Generation) для SAQ задач.

Показывает:
1. Загрузку индекса Wikipedia
2. Поиск релевантных документов по вопросу
3. Формирование промпта с контекстом

In [1]:
import os                                                                                                                                                                                                                         
os.environ['LD_LIBRARY_PATH'] = '/software/genoa/r24.04/bzip2/1.0.8-GCCcore-12.3.0/lib:' + os.environ.get('LD_LIBRARY_PATH', '')    

In [2]:
import sys
sys.path.insert(0, '../src')

from pathlib import Path
from qa_model.rag import WikipediaIndex, BM25Retriever
from qa_model.prompts import build_saq_prompt, SAQ_SYSTEM_PROMPT, SAQ_RAG_SYSTEM_PROMPT

ImportError: libbz2.so.1.0: cannot open shared object file: No such file or directory

## 1. Загрузка индекса Wikipedia

In [ ]:
# Путь к индексу (измените на свой)
INDEX_DIR = Path("/data/cat/ws/albu670g-qa-model/rag/wiki_index")

print(f"Loading index from: {INDEX_DIR}")
index = WikipediaIndex.load(INDEX_DIR)
print(f"Loaded {len(index):,} documents")

In [ ]:
# Создаём retriever
retriever = BM25Retriever(index=index, cache_enabled=False)

# Параметры RAG
TOP_K = 3
MAX_CONTEXT_TOKENS = 512

## 2. Тестовые вопросы

In [ ]:
# 5 примеров вопросов для SAQ
QUESTIONS = [
    "Who invented the telephone?",
    "What is the capital of France?",
    "When did World War II end?",
    "What is the chemical symbol for gold?",
    "Who wrote Romeo and Juliet?",
]

## 3. Демонстрация поиска и формирования промптов

Для каждого вопроса:
1. Выполняем BM25 поиск
2. Форматируем контекст
3. Строим финальный промпт

In [ ]:
def demo_rag_pipeline(question: str, retriever: BM25Retriever, top_k: int = 3, max_tokens: int = 512):
    """Демонстрация RAG pipeline для одного вопроса."""
    print("=" * 80)
    print(f"ВОПРОС: {question}")
    print("=" * 80)
    
    # 1. Поиск релевантных документов
    print(f"\n[1] BM25 поиск (top_k={top_k})...")
    documents = retriever.retrieve(question, top_k=top_k)
    
    print(f"\nНайдено {len(documents)} документов:")
    for i, doc in enumerate(documents, 1):
        title = doc.get('title', 'N/A')
        text_preview = doc.get('text', '')[:200] + "..."
        print(f"\n  [{i}] {title}")
        print(f"      {text_preview}")
    
    # 2. Форматирование контекста
    print(f"\n[2] Форматирование контекста (max_tokens={max_tokens})...")
    context = retriever.format_context(documents, max_tokens=max_tokens)
    
    print(f"\nКОНТЕКСТ ({len(context)} символов):")
    print("-" * 40)
    print(context)
    print("-" * 40)
    
    return context

In [ ]:
# Демо для всех вопросов
contexts = {}
for q in QUESTIONS:
    contexts[q] = demo_rag_pipeline(q, retriever, top_k=TOP_K, max_tokens=MAX_CONTEXT_TOKENS)
    print("\n")

## 4. Полные промпты с RAG контекстом

Показываем финальные промпты, которые отправляются в модель.

In [ ]:
# Загружаем tokenizer для форматирования промптов
from transformers import AutoTokenizer

# Используем LLaMA-3 tokenizer (или замените на свой)
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
print(f"Tokenizer loaded: {MODEL_ID}")

In [ ]:
def show_full_prompt(question: str, context: str, tokenizer):
    """Показать полный промпт с RAG контекстом."""
    print("=" * 80)
    print(f"ВОПРОС: {question}")
    print("=" * 80)
    
    # Промпт БЕЗ RAG
    prompt_no_rag = build_saq_prompt(tokenizer, question, context=None)
    
    # Промпт С RAG
    prompt_with_rag = build_saq_prompt(tokenizer, question, context=context)
    
    print("\n[БЕЗ RAG]")
    print("-" * 40)
    print(prompt_no_rag)
    print(f"\nТокенов: {len(tokenizer.encode(prompt_no_rag))}")
    
    print("\n[С RAG]")
    print("-" * 40)
    print(prompt_with_rag)
    print(f"\nТокенов: {len(tokenizer.encode(prompt_with_rag))}")
    
    return prompt_with_rag

In [ ]:
# Показать промпты для всех вопросов
for q in QUESTIONS:
    show_full_prompt(q, contexts[q], tokenizer)
    print("\n\n")

## 5. Сравнение System Prompts

In [ ]:
print("SAQ_SYSTEM_PROMPT (без RAG):")
print("-" * 40)
print(SAQ_SYSTEM_PROMPT)
print("\n")
print("SAQ_RAG_SYSTEM_PROMPT (с RAG):")
print("-" * 40)
print(SAQ_RAG_SYSTEM_PROMPT)

## 6. Интерактивный тест

Введите свой вопрос для проверки RAG pipeline.

In [ ]:
# Введите свой вопрос
custom_question = "What is the speed of light?"

# Поиск и контекст
custom_context = demo_rag_pipeline(custom_question, retriever, top_k=TOP_K, max_tokens=MAX_CONTEXT_TOKENS)

# Полный промпт
print("\n")
show_full_prompt(custom_question, custom_context, tokenizer)

## 7. Статистика индекса

In [ ]:
import os

print("Статистика индекса:")
print(f"  - Документов: {len(index):,}")

# Размеры файлов
docs_file = INDEX_DIR / "documents.json"
bm25_file = INDEX_DIR / "bm25_index.pkl"

if docs_file.exists():
    size_mb = docs_file.stat().st_size / (1024 * 1024)
    print(f"  - documents.json: {size_mb:.1f} MB")

if bm25_file.exists():
    size_mb = bm25_file.stat().st_size / (1024 * 1024)
    print(f"  - bm25_index.pkl: {size_mb:.1f} MB")

# Пример документа
print("\nПример документа из индекса:")
sample_doc = index.documents[0]
print(f"  Title: {sample_doc.get('title', 'N/A')}")
print(f"  Text: {sample_doc.get('text', '')[:300]}...")